In [1]:
import keras
import tensorflow as tf
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torch
from tqdm import tqdm
from torchvision.models.inception import Inception3
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from IPython.core.display import Image
from PIL import Image
import os
torch.manual_seed(0)

path = "/content/drive/MyDrive/VIP/data_road/" #Path of image data
train_path = path + "training/"
test_path = path + "testing/"

train_data = os.listdir(train_path + "image_2")
train_label = os.listdir(train_path + "gt_image_2")
test_data = os.listdir(test_path + "image_2")

class dataset(Dataset):
  def __init__(self,test=False,transf=transforms.ToTensor()):
    super().__init__()
    self.test=test
    self.trans = transf
    if test:
      self.data = test_data
    else:
      self.data = train_data
      self.labels = train_label

  def __len__(self):
      return len(self.data)

  def __getitem__(self, idx):
    if self.test:
      data_name = self.data[idx]
      data = Image.open(test_path + "image_2/" + data_name)
      data = data.resize((299, 299)) # Resize image first to fit dimension constraints on inception v3
      return self.trans(data), 0
    else:
      data_name = self.data[idx]
      label_name = self.labels[idx]
      data_img = Image.open(train_path + "image_2/" + data_name)
      label = Image.open(train_path + "gt_image_2/" + label_name)
      data_img = data_img.resize((299, 299)) 
      label = label.resize((299, 299)) 
      return self.trans(data_img), self.trans(label)

In [ ]:
def train_model(model, data_loader, loss, optimizer, epochs):
  device=torch.device('cuda:0') # Use GPU to run
  model.to(device)
  model.train()
  loss_list=[]
  b_list=[]
  for epoch in range(epochs):
    print("Epoch: ", (epoch + 1))
    model.train()
    for data, label in tqdm(train_data_load):
      data=data.to(device)
      label = label.to(device)
      output=model(data)
      batch_loss=loss(output, label)
      optimizer.zero_grad()
      batch_loss.backward()
      optimizer.step()
      b_list.append(batch_loss.cpu().detach().numpy())
    loss_list.append(sum(b_list)/len(b_list))
    print("Loss: ", sum(b_list)/len(b_list)) # Prints value of loss 
  return loss_list

In [ ]:
class model(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer2=torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
    self.MaxPool2b=nn.MaxPool2d(3,2)
    self.AvgPool=nn.AdaptiveAvgPool2d((1,1))
    self.Upsample=nn.Upsample(scale_factor=(1.17,1.035)) 

    self.t_layer1=nn.ConvTranspose2d(768,256,kernel_size=3,stride=5)
    self.t_layer2=nn.ConvTranspose2d(256,64,3,5)
    self.t_layer3=nn.Conv2d(64,32,3,3)
    self.t_layer4=nn.ConvTranspose2d(32,16,3,5)
    self.t_layer5=nn.Conv2d(16,3,3,2)
    self.t_layer6=nn.Conv2d(3,3,3)
    


  def forward(self,x):
    if self.layer2.transform_input:
        x_ch0 = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
        x_ch1 = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
        x_ch2 = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
        x = torch.cat((x_ch0, x_ch1, x_ch2), 1)
    # N x 3 x 299 x 299
    x = self.layer2.Conv2d_1a_3x3(x)
    # N x 32 x 149 x 149
    x = self.layer2.Conv2d_2a_3x3(x)
    # N x 32 x 147 x 147
    x = self.layer2.Conv2d_2b_3x3(x)
    # N x 64 x 147 x 147
    x = self.MaxPool2b(x)
    # N x 64 x 73 x 73
    x = self.layer2.Conv2d_3b_1x1(x)
    # N x 80 x 73 x 73
    x = self.layer2.Conv2d_4a_3x3(x)
    # N x 192 x 71 x 71
    x = self.MaxPool2b(x)
    # N x 192 x 35 x 35
    x = self.layer2.Mixed_5b(x)
    # N x 256 x 35 x 35
    x = self.layer2.Mixed_5c(x)
    # N x 288 x 35 x 35
    x = self.layer2.Mixed_5d(x)
    # N x 288 x 35 x 35
    x = self.layer2.Mixed_6a(x)
    # N x 768 x 17 x 17
    x = self.layer2.Mixed_6b(x)
    # N x 768 x 17 x 17
    x = self.layer2.Mixed_6c(x)
    # N x 768 x 17 x 17
    x = self.layer2.Mixed_6d(x)
    # N x 768 x 17 x 17
    x = self.layer2.Mixed_6e(x)
    
    # N x 256 x 85 x 85
    x=self.t_layer1(x)
    # N x 64 x 413 x 413 
    x=self.t_layer2(x)
    # N x 32 x 137 x 137 
    x=self.t_layer3(x)
    # N x 16 x 683 x 683 
    x=self.t_layer4(x)
    # N x 3 x 341 x 341
    x=self.t_layer5(x)
    # N x 3 x 299 x 299
    for i in range(21):
      x=self.t_layer6(x)

    return x



model=model()
loss=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters())

train_data_obj=dataset()
train_data_load=DataLoader(train_data_obj, batch_size=16, shuffle=True, drop_last=True)

loss_list=train_model(model, train_data_load, loss, optimizer, epochs=50) #Change number of epochs to train model


In [ ]:
model.eval()

test_data_obj=dataset(test=True)
test_data_loader=DataLoader(test_data_obj, batch_size=16, shuffle=True)

for data, _ in test_data_loader:
  device=torch.device('cuda:0')
  data=data.to(device)
  predictions=model(data)
  break

In [ ]:
# Show superimposed image
for i in range(10):
  import matplotlib.pyplot as plt
  pred=predictions[i].permute(1,2,0)
  plt.imshow(pred.cpu().detach().numpy())

  test_img=data[i].permute(1,2,0)
  plt.imshow(test_img.cpu().detach().numpy(), alpha = 0.6)
  plt.show()